# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [8]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

#nltk for tokenizing/lemmatizing
import nltk
nltk.download(['punkt','wordnet'])



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ben\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ben\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
# load data from database
engine = create_engine('sqlite:///textmessages.db')
df = pd.read_sql_table('MessageTable',con=engine)

In [18]:
X = df['message']
y = df.iloc[:,3:]

In [19]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [20]:
y.head()

,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,direct,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,direct,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,direct,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,direct,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,direct,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2. Write a tokenization function to process your text data

In [21]:
#this function will be used later as the tokenizer param in the CountVectorizer 
def tokenize(text):
    #split into tokens
    tokens = nltk.word_tokenize(text)
    
    #get to the ROOT of the words (lemmatize)
    lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
    
    clean_tokens = []
    for token in tokens:
        clean_token = lemmatizer.lemmatize(token, pos='v').lower()
        clean_tokens.append(clean_token)
    
    return clean_tokens

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [22]:
# X[X.loc[X_train['message'].index].columns[2:]]

In [85]:
class Grab_Categories(BaseEstimator, TransformerMixin):
    def __init__(self, data):
        self.data = data
    
    def grab_cats(self, message):
        mes_idx = message.index
        
        return data[data.loc[data[mes_idx].columns[2:]]]
    
    def fit(self, X):
        return self
    
    def transform(self, X):
        X_cats = pd.Series(X).apply(self.grab_cats)
        return pd.DataFrame(X_cats)

In [26]:
#works purely on message column
pipeline = Pipeline(
    [('vectorizer',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier))]
)

# pipeline = Pipeline([
#     ('features',FeatureUnion[
#         #performs text tokenizing/vectorizing/TfidfTransform on messages
#         ('text_pipeline',Pipeline([
#             ('vectorizer',CountVectorizer(tokenizer=tokenize)),
#             ('tfidf',TfidfTransformer())
#         ])),
#         #grabs categories previously provided for messages
#         ('grab_categories',Grab_Categories(data=X))
#     ]),
#     ('clf',RandomForestClassifier(random_state=42))
# ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [21]:
y_train.shape

(21076,)

In [28]:
pipeline.fit(X_train, y_train)

TypeError: '>' not supported between instances of 'float' and 'str'

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.